In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import stft, istft
from google.colab import drive

# 🔹 Montar Google Drive
drive.mount('/content/drive')

# 🔹 Ruta del archivo en Google Drive
audio_path = "/content/drive/MyDrive/Data_Augmentation/JUAN/AUDIO_DATA_AUGM/DATA/VNE_31_P2_42_BI.wav"

# 🔹 Cargar el archivo de audio
sample_rate, audio = wavfile.read(audio_path)

# 🔹 Verificar si el audio es estéreo o mono
is_stereo = len(audio.shape) > 1
if is_stereo:
    left_channel = audio[:, 0]  # Canal izquierdo
    right_channel = audio[:, 1]  # Canal derecho
else:
    left_channel = right_channel = audio  # Si es mono, tratamos ambos como el mismo

# 🔹 Función para aplicar Pitch Shift usando Phase Vocoder a un solo canal
def pitch_shift(audio, sample_rate, semitones):
    factor = 2 ** (semitones / 12)  # Factor de cambio de tono
    window_size = 1024
    hop_size = window_size // 4

    # Calcular la STFT
    f, t, Zxx = stft(audio, fs=sample_rate, nperseg=window_size, noverlap=hop_size)

    # Validación para evitar división por cero
    if np.max(np.abs(Zxx)) == np.min(np.abs(Zxx)):
        print("⚠️ La STFT tiene valores uniformes, posible error en la señal.")
        return audio

    # Ajustar la frecuencia multiplicando por el factor de cambio de tono
    new_f = f * factor

    # Interpolación correcta
    shifted_Zxx = np.zeros_like(Zxx, dtype=np.complex_)
    for i in range(len(t)):
        shifted_Zxx[:, i] = np.interp(f, new_f, np.abs(Zxx[:, i]), left=0, right=0) * np.exp(1j * np.angle(Zxx[:, i]))

    # Reconstrucción de la señal con la iSTFT
    _, shifted_audio = istft(shifted_Zxx, fs=sample_rate, nperseg=window_size, noverlap=hop_size)

    return shifted_audio.astype(np.int16)

# 🔹 Aplicar Pitch Shift a cada canal por separado (+3 semitonos)
shifted_left = pitch_shift(left_channel, sample_rate, semitones=3)
shifted_right = pitch_shift(right_channel, sample_rate, semitones=3)

# 🔹 Ajustar tamaños si hay desajuste de longitud
min_length = min(len(shifted_left), len(shifted_right))
shifted_left, shifted_right = shifted_left[:min_length], shifted_right[:min_length]

# 🔹 Combinar ambos canales en una señal estéreo
shifted_audio_stereo = np.column_stack((shifted_left, shifted_right))

# 🔹 Guardar el archivo modificado en Google Drive
output_path = "/content/drive/MyDrive/Data_Augmentation/JUAN/AUDIO_DATA_AUGM/AudioPuro/pp_pitch_shift_stereo.wav"
wavfile.write(output_path, sample_rate, shifted_audio_stereo)

# 🔹 PLOTEAR GRÁFICAS CON ETIQUETAS

# 1️⃣ Señales en el dominio del tiempo
fig, axs = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

#✅ ¿Qué hace?:
#plt.figure() crea una ventana para la gráfica.
#figsize=(12, 4) ajusta el tamaño (12 pulgadas de ancho y 4 pulgadas de alto)✅.

time_original = np.linspace(0, len(left_channel) / sample_rate, num=len(left_channel))
time_shifted = np.linspace(0, len(shifted_left) / sample_rate, num=len(shifted_left))

#✅ Usa np.linspace() para crear una línea de tiempo desde 0 segundos hasta la duración total del audio
# duracion (s) = numero de muestras/frecuencia de muestreo ✅.

axs[0].plot(time_original[:1000], left_channel[:1000], label="Canal Izquierdo Original")
axs[0].plot(time_shifted[:1000], shifted_left[:1000], label="Canal Izquierdo Pitch Shifted", linestyle="dashed")
axs[0].set_ylabel("Amplitud")
axs[0].set_title("Comparación del Canal Izquierdo en el Dominio del Tiempo")
axs[0].legend()
axs[0].grid()

axs[1].plot(time_original[:1000], right_channel[:1000], label="Canal Derecho Original")
axs[1].plot(time_shifted[:1000], shifted_right[:1000], label="Canal Derecho Pitch Shifted", linestyle="dashed")
axs[1].set_xlabel("Tiempo (s)")
axs[1].set_ylabel("Amplitud")
axs[1].set_title("Comparación del Canal Derecho en el Dominio del Tiempo")
axs[1].legend()
axs[1].grid()

plt.show()

# 2️⃣ Espectrograma de la señal original
plt.figure(figsize=(12, 5))
f, t, Zxx = stft(left_channel, fs=sample_rate, nperseg=1024, noverlap=256)
plt.pcolormesh(t, f, np.log1p(np.abs(Zxx)), shading='gouraud')  # log1p mejora la visualización
plt.xlabel("Tiempo (s)")
plt.ylabel("Frecuencia (Hz)")
plt.title("Espectrograma del Canal Izquierdo Original")
plt.colorbar(label="Magnitud")
plt.show()

# 3️⃣ Espectrograma del Canal Izquierdo con Pitch Shift
plt.figure(figsize=(12, 5))
f, t, Zxx = stft(shifted_left, fs=sample_rate, nperseg=1024, noverlap=256)
plt.pcolormesh(t, f, np.log1p(np.abs(Zxx)), shading='gouraud')
plt.xlabel("Tiempo (s)")
plt.ylabel("Frecuencia (Hz)")
plt.title("Espectrograma del Canal Izquierdo Pitch Shifted")
plt.colorbar(label="Magnitud")
plt.show()

# 4️⃣ Espectrograma del Canal Derecho con Pitch Shift
plt.figure(figsize=(12, 5))
f, t, Zxx = stft(shifted_right, fs=sample_rate, nperseg=1024, noverlap=256)
plt.pcolormesh(t, f, np.log1p(np.abs(Zxx)), shading='gouraud')
plt.xlabel("Tiempo (s)")
plt.ylabel("Frecuencia (Hz)")
plt.title("Espectrograma del Canal Derecho Pitch Shifted")
plt.colorbar(label="Magnitud")
plt.show()

# 🔹 Gráfica de la señal original con ambos canales
plt.figure(figsize=(12, 4))

# Crear eje de tiempo
time_original = np.linspace(0, len(left_channel) / sample_rate, num=len(left_channel))

# Graficar los dos canales
plt.plot(time_original[:1000], left_channel[:1000], label="Canal Izquierdo Original", alpha=0.8)
plt.plot(time_original[:1000], right_channel[:1000], label="Canal Derecho Original", linestyle="dashed", alpha=0.8)

#✅ ¿Qué hace?:
# Usa plt.plot() para dibujar las formas de onda.
# time_original[:1000]: Solo muestra los primeros 1000 puntos para no saturar la gráfica.
# label="...": Pone nombres a las líneas.
# alpha=0.8: Hace las líneas ligeramente transparentes para ver mejor la superposición.
# El Canal Derecho aparece como línea punteada (linestyle="dashed") para distinguirlos✅.

# Etiquetas y formato
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.title("Señal Estéreo Original (Antes de Pitch Shift)")
plt.legend()
plt.grid()
plt.show()

# 5️⃣ Señal estéreo combinada en el dominio del tiempo
plt.figure(figsize=(12, 4))
plt.plot(time_shifted[:1000], shifted_left[:1000], label="Canal Izquierdo Pitch Shifted")
plt.plot(time_shifted[:1000], shifted_right[:1000], label="Canal Derecho Pitch Shifted", linestyle="dashed")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.title("Señal Estéreo Final (Ambos Canales Combinados)")
plt.legend()
plt.grid()
plt.show()

# 🔹 Mostrar información final
print(f"✅ Archivo estéreo modificado guardado en: {output_path}")
print("🔍 Primeros 20 valores después de la transformación (Canal Izquierdo):", shifted_left[:20])
print("🔍 Primeros 20 valores después de la transformación (Canal Derecho):", shifted_right[:20])